25 additional ML-flavoured coding challenges (Python)

Designed to be solvable in ~10–20 minutes each; most require only core libs (math, collections, heapq, etc.).



ROC Points: Given scores+labels, produce ROC curve points (sorted by score); compute AUC via trapezoids.

PR-AUC: As above but Precision–Recall AUC; careful with ties and stepwise areas.

Balanced Accuracy & MCC: Compute Balanced Accuracy and Matthew’s Correlation from counts.

Stratified Split: Split (X, y) into train/val with class-ratio preserved; return indices.

K-Fold Indexer: Produce k folds (list of (train_ids, val_ids)) with near-equal sizes.

Online Mean/Variance: Welford’s algorithm to stream-compute mean and variance.

Exponential Moving Average: Implement EMA for a series (alpha given).

Time-window Aggregates: Given events (ts, value), compute rolling sum over last W seconds (use deque).

Cosine Similarity: Given two sparse vectors as dicts, compute cosine similarity efficiently.

TF-IDF (mini): Given list of docs (token lists), compute tf-idf for each term in one target doc.

K-Means (one iteration): Given points and current centroids, assign & recompute new centroids.

Softmax + Cross-Entropy: Numerically stable softmax; compute average CE for logits+labels.

Sigmoid + BCE: Stable sigmoid and binary cross-entropy with epsilon for extremes.

Top-K by Score: Stream of (id, score) → maintain top-k using a min-heap.

Deduplicate by Key: From records with id, keep the one with max timestamp; return stable order by time.

Sessionize Clicks: Group user events into sessions: new session if gap > T minutes.

Log Latency SLOs: From microservice logs, compute p50/p90/p99 latencies per route; output failing routes vs SLO.

LRU Cache: Classic LRU with O(1) get/put (OrderedDict or custom DLL + hash).

Tokenize + N-grams: Clean text (lower/strip punctuation) and output counts of n-grams (n given).

Levenshtein Distance: Edit distance DP; return distance & one alignment path.

Vector Search (Brute): Given query vector and corpus, return top-k by cosine; show complexity.

Min Hash-like Signature (toy): Compute simple k hash functions on token set to build a signature vector.

A/B Results Calc: Given conversions & trials for A and B, compute absolute & relative lift and a z-test p-value.

Feature Scaling: Implement standard scaling (z-score) and min–max scaling with fit/transform API.

## Problem #1

Confusion Matrix Metrics: Given TP, FP, TN, FN, compute Precision, Recall, F1 (handle zero-div).

Step 1: Assume input as 4 integers for 
    TP = 50, FP=10, TN=80, FN=5

Step 2: Compute Precision, Recall, and F1 Score, rounded to 4 decimal places.

Precision = TP / (TP+FP) 
Recall = TP / (TP+FN)
F1 = 2PR / (P+R) # is the harmonic Mean
If any denominator is zero, return 0.0 for that metric.

Step 3: Output: three floats rounded to 4 decimals, space-separated.

In [ ]:
# read inputs 
# compute

def evalModelScore():
    # TP, FP, TN, FN = map (int, input().split())
    TP, FP, TN, FN = 50, 30, 60, 10
    
    def safeDiv(num, den):
        return num/den if den else 0.0
      
    precision = safeDiv(TP, (TP + FP))
    recall =safeDiv(TP, (TP + FN))
    f1 = (2 * precision * recall) / (precision + recall)

    print (f"{precision:.4f}, {recall:.4f}, {f1:.4f}")

if __name__ == "__main__":
    evalModelScore() 



0.6250, 0.8333, 0.7143


In [ ]:
# If you are given an array of predictions and labels (0, 1), you can compute TP/FP/TN/FN in one pass 

def evalModel():
    n = input(input())
    preds = list(map(int, input().split()))
    labels = list(map(int, input().split()))

    TP = sum(p==1 and l==1 for p,l in zip(preds,labels))
    FP = sum(p==1 and l==0 for p,l in zip(preds,labels))
    TN = sum(p==0 and l==0 for p,l in zip(preds,labels))
    FN = sum(p==0 and l==1 for p,l in zip(preds,labels))

    def safe_div(a,b): return a/b if b else 0.0
    P = safe_div(TP, TP+FP); R = safe_div(TP, TP+FN)
    F1 = safe_div(2*P*R, P+R)
    print(f"{P:.4f} {R:.4f} {F1:.4f}")

if __name__ == "__main__":
    evalModel()



## Problem #2 

Given predicted scores (higher ⇒ more positive) and binary labels (0/1), output the ROC points (FPR, TPR) as the threshold sweeps from +∞ → −∞, and compute ROC-AUC (trapezoidal rule).

Include the anchors (0,0) and (1,1).

Handle ties in scores correctly.

If there are no positives or no negatives, AUC is 0.0 by convention (or return just the points).



12-minute plan

0–2 min — Frame it

Sort examples by score descending.

Let P = #positives, N = #negatives.

Sweep through sorted items; at each distinct score, move the threshold below that score (i.e., items at that score flip to positive).

Maintain cumulative TP and FP.

After each distinct score, append a ROC point:

TPR = TP / P, FPR = FP / N.

Start with (0,0), finish with (1,1).

AUC = Σ trapezoids along FPR.

2–7 min — Code it (clean + safe)

Group by score to handle ties in O(n).

Protect divisions when P==0 or N==0.

7–10 min — Test quickly

Tiny examples (all positives, all negatives, simple mixed).

10–12 min — Tidy & submit

Ensure formatting and edge-case branches are sound.

In [ ]:
def roc_points_and_auc(scores, labels):
    # Pair and sort by score (desc)
    data = sorted(zip(scores, labels), key=lambda x: x[0], reverse=True)
    P = sum(labels)
    N = len(labels) - P
    if P == 0 or N == 0:
        # Degenerate ROC; return anchors and zero AUC
        return [(0.0, 0.0), (1.0, 1.0)], 0.0

    points = [(0.0, 0.0)]  # (FPR, TPR)
    TP = FP = 0

    i = 0
    while i < len(data):
        # Process all items with the same score (tie group)
        j = i
        # Count how many positives/negatives in this tie group
        pos_in_grp = neg_in_grp = 0
        score_i = data[i][0]
        while j < len(data) and data[j][0] == score_i:
            if data[j][1] == 1:
                pos_in_grp += 1
            else:
                neg_in_grp += 1
            j += 1

        # Move threshold past this score => these become predicted positive
        TP += pos_in_grp
        FP += neg_in_grp

        # Record ROC point AFTER applying the whole tie group
        TPR = TP / P
        FPR = FP / N
        points.append((FPR, TPR))

        i = j

    # Ensure (1,1) present
    if points[-1] != (1.0, 1.0):
        points.append((1.0, 1.0))

    # Trapezoidal AUC along FPR axis
    auc = 0.0
    for k in range(1, len(points)):
        x0, y0 = points[k-1]
        x1, y1 = points[k]
        auc += (x1 - x0) * (y0 + y1) / 2.0

    return points, auc


# Minimal I/O wrapper (choose one input style)
def solve():
    # Example input style:
    # n
    # scores (space-separated floats)
    # labels (space-separated 0/1)
    import sys
    it = iter(sys.stdin.read().strip().split())
    try:
        n = int(next(it))
    except StopIteration:
        return
    scores = [float(next(it)) for _ in range(n)]
    labels = [int(next(it)) for _ in range(n)]
    pts, auc = roc_points_and_auc(scores, labels)
    # Print points then AUC (format to 4 decimals)
    for fpr, tpr in pts:
        print(f"{fpr:.4f} {tpr:.4f}")
    print(f"AUC {auc:.4f}")


if __name__ == "__main__":
    solve()
    # Comment out solve() if Canditech provides function signature only
    # and calls your function via tests.
    # For local sanity:
    # Example:
    # n=6
    # scores: 0.9 0.8 0.8 0.7 0.1 0.0
    # labels:   1   0   1   0   1   0
    # Expected monotonic ROC from (0,0) to (1,1), AUC ~ 0.75
    # You can quickly sanity-check by piping input.
    pass


Reasoning

Sorted by score (already sorted).

P=3 (labels=1), N=3.

After 0.9 group: TP=1, FP=0 → (FPR=0/3=0.0, TPR=1/3≈0.3333).

After 0.8 group (tie of 2: labels 0,1): TP=2, FP=1 → (0.3333, 0.6667).

After 0.7 group (label 0): TP=2, FP=2 → (0.6667, 0.6667).

After 0.1 group (label 1): TP=3, FP=2 → (0.6667, 1.0000).

After 0.0 group (label 0): TP=3, FP=3 → (1.0000, 1.0000).
AUC ≈ 0.75.

⚠️ Pitfalls you’ll avoid

Not grouping ties → jagged, wrong AUC.

Forgetting anchors (0,0) and (1,1).

Division by zero when all labels are 0 or 1.

Sorting ascending by mistake.

Printing too many decimals or wrong format.

⏩ Speed tips

Precompute P, N; early-return if degenerate.

Group ties with a simple while (faster than itertools.groupby in contest settings).

Keep only (FPR,TPR); you don’t need to store thresholds.

## Problem #3

